In [ ]:
import pandas as pd
from itertools import combinations

In [ ]:
# Load the dataset
file_path = '/content/drive/MyDrive/dataset/pipe_small_dataset.xlsx'  # Replace with your file path
dataset = pd.read_excel(file_path)

In [ ]:
# Load the dataset
file_path = '/content/drive/MyDrive/dataset/fitting_small_dataset.csv'  # Replace with your file path
dataset = pd.read_csv(file_path)

In [ ]:
# Function to generate all combinations of column-value pairs for a row
def generate_all_combinations(row, columns):
    """
    Generate all possible combinations of column values for a row.
    For each combination, create a question and answer pair.
    """
    question_answer_pairs = []

    # Loop over combinations of 1, 2, 3, ... up to all columns
    for i in range(1, len(columns) + 1):
        # Get all combinations of columns of size 'i'
        for combo in combinations(columns, i):
            question_parts = []  # List to hold the parts of the question
            answer_parts = []    # List to hold the parts of the answer

            # Loop over each column in the current combination
            for col in combo:
                # Get the value from the row for the current column
                value = row[col]

                # Check if the value is not null
                if pd.notnull(value):
                    # Add the value to the question list
                    question_parts.append(str(value))

                    # Add the 'COLUMN_NAME: value' format to the answer list
                    answer_parts.append(f"{col}: {value}")

            # Join the question and answer parts with spaces and commas, respectively
            question = " ".join(question_parts)
            answer = ", ".join(answer_parts)

            # Add the complete question-answer pair to the list
            question_answer_pairs.append((question, answer))

    return question_answer_pairs  # Return all generated question-answer pairs






In [ ]:
# Initialize a list to store data for all rows
all_combinations_list = []

# Process each row in the dataset
for row_index, row in dataset.iterrows():
    # Generate all combinations for the current row
    combinations_list = generate_all_combinations(row, dataset.columns)

    # Create an ID for each combination and add to the list
    for idx, (question, answer) in enumerate(combinations_list):
        all_combinations_list.append(
            {"id": f"{row_index + 1}_{idx + 1}", "Question": question, "Answer": answer}
        )

# Convert the list of dictionaries into a DataFrame
all_combinations_data = pd.DataFrame(all_combinations_list)

check the error for large data which is out of bound for sheet

In [ ]:
# # Save the resulting dataset to a new Excel file
# output_file = '/content/drive/MyDrive/dataset/all_combinations_for_fitting_all_rows.xlsx'  # Replace with your desired path
# all_combinations_data.to_excel(output_file, index=False)

# print(f"Combinations file saved to {output_file}")

In [ ]:
# Save the resulting dataset to a new Excel file
output_file = '/content/drive/MyDrive/dataset/all_combinations_for_fitting_all_rows.csv'  # Replace with your desired path
all_combinations_data.to_csv(output_file, index=False)

print(f"Combinations file saved to {output_file}")

Combinations file saved to /content/drive/MyDrive/dataset/all_combinations_for_fitting_all_rows.csv


In [ ]:
!pip install transformers nltk nlpaug augly

In [ ]:
import nltk
from nltk.corpus import wordnet
import random
import pandas as pd
from transformers import pipeline, MarianMTModel, MarianTokenizer
import nlpaug.augmenter.word as naw
import augly.text as txt

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# # Load dataset (replace 'fixed_merged_training_data.csv' with your file path)
# df = pd.read_excel('/content/drive/MyDrive/dataset/all_combinations_for_fitting_all_rows.xlsx')

# Load dataset (replace 'fixed_merged_training_data.csv' with your file path)
df = pd.read_csv('/content/drive/MyDrive/dataset/all_combinations_for_fitting_all_rows.csv')


In [ ]:
# Function for Synonym Replacement
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()
    for _ in range(n):
        word_to_replace = random.choice(new_words)
        synonyms = wordnet.synsets(word_to_replace)
        if synonyms:
            lemmas = synonyms[0].lemmas()
            new_word = random.choice(lemmas).name()
            new_words = [new_word if word == word_to_replace else word for word in new_words]
    return ' '.join(new_words)

# Function for Random Swap
def random_swap(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Function for Random Deletion
def random_deletion(sentence, p=0.1):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    new_words = [word for word in words if random.uniform(0, 1) > p]
    return ' '.join(new_words) if new_words else random.choice(words)

# Combined Augmentation Function
def combined_augmentation(sentence):
    # Apply synonym replacement
    sentence = synonym_replacement(sentence, n=2)
    # Apply random swap
    sentence = random_swap(sentence, n=1)
    # Apply random deletion
    sentence = random_deletion(sentence, p=0.1)
    return sentence

# Example usage of combined augmentation
sentence = "This is a sample sentence for augmentation."
augmented_sentence = combined_augmentation(sentence)
print("Original:", sentence)
print("Augmented:", augmented_sentence)


Original: This is a sample sentence for augmentation.
Augmented: This for sample a augmentation.


In [ ]:
augmented_data = []
for _, row in df.iterrows():
    original_question = row['Question']

    # Check if the question is a valid string
    if isinstance(original_question, str):
        # Try to generate augmented questions
        augmented_questions = []
        try:
            augmented_questions = [combined_augmentation(original_question) for _ in range(10)]
        except ValueError as e:
            print(f"Augmentation failed for question: {original_question} with error: {e}")
            augmented_questions = [original_question]  # Use original question as a fallback
    else:
        print(f"Invalid question (not a string): {original_question}")
        augmented_questions = [str(original_question)]  # Convert NaN/None to string if necessary

    # Append the augmented data
    for augmented_question in augmented_questions:
        augmented_data.append({
            'id': row['id'],
            'Question': augmented_question,
            'Answer': row['Answer']
        })


Streaming output truncated to the last 5000 lines.
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or is negative
Augmentation failed for question: SMLS with error: Sample larger than population or i

In [ ]:
# Convert to DataFrame and save
augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv('augmented_fitting_training_data.csv', index=False)

#avoid gasket.


NameError: name 'augmented_data' is not defined

In [ ]:
augmented_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17616 entries, 0 to 17615
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        17616 non-null  object
 1   Question  17616 non-null  object
 2   Answer    17615 non-null  object
dtypes: object(3)
memory usage: 413.0+ KB


In [ ]:
augmented_df.head()

,id,Question,Answer
0,1_1,RJG02R9004,BASE_ITEM_CODE: RJG02R9004
1,1_2,ring gasket JOINT,CATEGORY: RING JOINT GASKET
2,1_2,articulation RING gasket,CATEGORY: RING JOINT GASKET
3,1_2,JOINT ring gasket,CATEGORY: RING JOINT GASKET
4,1_2,ring GASKET articulation,CATEGORY: RING JOINT GASKET


In [ ]:
augmented_df.tail()

,id,Question,Answer
17611,7_255,SYNTHETIC SYNTHETIC fiber GASKET 1 CLS 150 FUL...,"BASE_ITEM_CODE: SFG01R1001, CATEGORY: SYNTHETI..."
17612,7_255,SYNTHETIC gasket FIBER FULL FACE 33.4 B16.5/AS...,"BASE_ITEM_CODE: SFG01R1001, CATEGORY: SYNTHETI..."
17613,7_255,SFG01R1001 synthetic_substance FIBER GASKET CL...,"BASE_ITEM_CODE: SFG01R1001, CATEGORY: SYNTHETI..."
17614,7_255,SFG01R1001 SYNTHETIC FIBER SYNTHETIC CLS 150 F...,"BASE_ITEM_CODE: SFG01R1001, CATEGORY: SYNTHETI..."
17615,7_255,SFG01R1001 SYNTHETIC FIBER GASKET ace full-of-...,"BASE_ITEM_CODE: SFG01R1001, CATEGORY: SYNTHETI..."


In [ ]:
from google.colab import files
files.download('augmented_training_data.csv')

In [ ]:
# Load the augmented dataset
augmented_df = pd.read_csv('augmented_fitting_training_data.csv')

# Fix the id column to be sequential starting from 1
augmented_df['id'] = range(1, len(augmented_df) + 1)

# Save the corrected dataframe
augmented_df.to_csv('corrected_augmented_fitting_training_data.csv', index=False)

# Download the corrected dataset
from google.colab import files
files.download('corrected_augmented_fitting_training_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Prompt user to input the full paths for four CSV files
file_paths = ['/content/drive/MyDrive/dataset/corrected_augmented_training_data_final_only_for_pipe.csv', '/content/drive/MyDrive/dataset/corrected_augmented_valve_training_data.csv', '/content/drive/MyDrive/dataset/corrected_augmented_gasket_training_data.csv']
# for i in range(4):
#     file_path = input(f"Enter the full path for file {i+1} (e.g., /path/to/file.csv): ")
#     file_paths.append(file_path)

# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through each file path, read it into a DataFrame, and append to the list
for path in file_paths:
    df = pd.read_csv(path)
    dataframes.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a new CSV file
output_path = "/content/drive/MyDrive/dataset/corrected_augmented_merged_training_data.csv"
merged_df.to_csv(output_path, index=False)

print(f"Merged CSV file saved at: {output_path}")


Merged CSV file saved at: /content/drive/MyDrive/dataset/corrected_augmented_merged_training_data.csv
